<a href="https://colab.research.google.com/github/switch-kosuke/Udemy_DataAnalysis/blob/main/model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import glob
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
#import test_func as tf
%matplotlib inline

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
files = glob.glob("/content/drive/MyDrive/Colab Notebooks/data_comp/train/train/*.csv")
data_list = []
for file in files:
    data_list.append(pd.read_csv(file, index_col=0))
df = pd.concat(data_list)

<ipython-input-41-dd1732534ad3>:4: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  data_list.append(pd.read_csv(file, index_col=0))
<ipython-input-41-dd1732534ad3>:4: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  data_list.append(pd.read_csv(file, index_col=0))


In [ ]:
#df = tf.data_pre(df)
#model = tf.model_lgb(df)

NameError: ignored

In [ ]:
#edaでやったことを、関数化する
def data_pre(df):
    nonnull_list = []
    for col in df.columns:
        nonnull = df[col].count()
        if nonnull == 0:
            nonnull_list.append(col)
    df = df.drop(nonnull_list, axis=1)

    df = df.drop("市区町村名", axis=1)

    df = df.drop("種類", axis=1)

    dis = {
        "30分?60分":45,
        "1H?1H30":75,
        "2H?":120,
        "1H30?2H":105
    }
    df["最寄駅：距離（分）"] = df["最寄駅：距離（分）"].replace(dis).astype(float)

    df["面積（㎡）"] = df["面積（㎡）"].replace("2000㎡以上", 2000).astype(float)


    y_list = {}
    for i in df["建築年"].value_counts().keys():
        if "平成" in i:
            num = float(i.split("平成")[1].split("年")[0])
            year = 33 - num
        if "令和" in i:
            num = float(i.split("令和")[1].split("年")[0])
            year = 3 - num
        if "昭和" in i:
            num = float(i.split("昭和")[1].split("年")[0])
            year = 96 - num
        y_list[i] = year
    y_list["戦前"] = 76
    df["建築年"] = df["建築年"].replace(y_list)

    year = {
        "年第１四半期": ".25",
        "年第２四半期": ".50",
        "年第３四半期": ".75",
        "年第４四半期": ".99"
    }
    year_list = {}
    for i in df["取引時点"].value_counts().keys():
        for k, j in year.items():
            if k in i:
                year_rep = i.replace(k, j)
        year_list[i] = year_rep
    df["取引時点"] = df["取引時点"].replace(year_list).astype(float)

    #LightGBMでは、カテゴリ質的変数であることを書く必要がある
    for col in ["都道府県名", "地区名", "最寄駅：名称", "間取り", "建物の構造", "用途", "今後の利用目的", "都市計画", "改装", "取引の事情等"]:
                df[col] = df[col].astype("category")

    return df

In [ ]:
df = data_pre(df)
df.info()

In [ ]:
import lightgbm as lgb #LightGBMの作成
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error as mae

df_train, df_val =train_test_split(df, test_size=0.2)

col = "取引価格（総額）_log"
train_y = df_train[col] #学習用の目的変数
train_x = df_train.drop(col, axis=1) #学習用の説明変数

val_y = df_val[col] #検証用の目的変数
val_x = df_val.drop(col, axis=1) #検証用の説明変数

#LightGBM用のデータセットに変換
trains = lgb.Dataset(train_x, train_y)
valids = lgb.Dataset(val_x, val_y)

params = {
    "objective": "regression", #回帰のタスク
    "metrics": "mae" #平均絶対誤差 mae
}

#num_boost_round 決定木の回数 early_stopping_rounds　過学習を防ぐために早めに停止する
model = lgb.train(params, trains, valid_sets=valids, num_boost_round=1000, callbacks=[lgb.early_stopping(stopping_rounds=100)])

In [ ]:
#平均絶対誤差で評価
vals = model.predict(val_x)
mae(vals, val_y)

In [ ]:
df_test = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/data_comp/test.csv", index_col=0)
#テストデータをきれいにする
df_test = data_pre(df_test)

In [ ]:
predict = model.predict(df_test)
df_test["取引価格（総額）_log"] = predict
df_test[["取引価格（総額）_log"]].to_csv("/content/drive/MyDrive/Colab Notebooks/data_comp/submit_test.csv") #目的変数の予測結果をcsvに

In [ ]:
df.info()

In [ ]:
#特徴量重要度を表示する
pd.DataFrame(model.feature_importance(), index=val_x.columns, columns=["importance"]).sort_values("importance", ascending=False)